In [1]:
from hybridSVD import hsvd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# !wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
# !unzip  "ml-1m.zip" -d data
# !rm  "ml-1m.zip"


In [3]:
ratings = pd.read_csv('data/ml-1m/ratings.dat', sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])

/tmp/ipykernel_127033/547913287.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('data/ml-1m/ratings.dat', sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])


In [13]:
movie_features = pd.read_csv('data/ml-1m/movies.dat', sep='::',names=['movieid', 'title', 'genres'],
                              encoding='ascii',encoding_errors='replace') # some issue in line 3114

/tmp/ipykernel_127033/542126174.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movie_features = pd.read_csv('data/ml-1m/movies.dat', sep='::',names=['movieid', 'title', 'genres'],


In [ ]:
#encode genres
movie_features['genres'] = movie_features['genres'].apply(lambda x: str(x).split('|'))
all_genres = set(list(movie_features['genres'].sum()))
for genre in all_genres:
    movie_features[genre] = movie_features['genres'].apply(lambda x: genre in x).astype(int)

In [23]:
user_features = pd.read_csv('data/ml-1m/users.dat', sep='::', names=['userid', 'gender', 'age', 'occupation', 'zip-code'])

/tmp/ipykernel_127033/1697848501.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user_features = pd.read_csv('data/ml-1m/users.dat', sep='::', names=['userid', 'gender', 'age', 'occupation', 'zip-code'])


In [24]:
occupations_dict = {
    0:  "other",
    1:  "academic/educator",
    2:  "artist",
    3:  "clerical/admin",
    4:  "college/grad student",
    5:  "customer service",
    6:  "doctor/health care",
    7:  "executive/managerial",
    8:  "farmer",
    9:  "homemaker",
    10:  "K-12 student",
    11:  "lawyer",
    12:  "programmer",
    13:  "retired",
    14:  "sales/marketing",
    15:  "scientist",
    16:  "self-employed",
    17:  "technician/engineer",
    18:  "tradesman/craftsman",
    19:  "unemployed",
    20:  "writer"
}
user_features['occupation'] = user_features['occupation'].apply(lambda x: occupations_dict[x])
occupations_dummies = pd.get_dummies(user_features['occupation'])
age_dummies = pd.get_dummies(user_features['age'],prefix='age')
user_features['gender'] = user_features['gender'].apply(lambda x: 1 if x=='M' else 0)
user_features = pd.concat([user_features, occupations_dummies, age_dummies], axis=1)


In [25]:
user_features

,userid,gender,age,occupation,zip-code,K-12 student,academic/educator,artist,clerical/admin,college/grad student,...,tradesman/craftsman,unemployed,writer,age_1,age_18,age_25,age_35,age_45,age_50,age_56
0,1,0,1,K-12 student,48067,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2,1,56,self-employed,70072,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,1,25,scientist,55117,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1,45,executive/managerial,02460,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,5,1,25,writer,55455,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,0,25,scientist,32603,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6036,6037,0,45,academic/educator,76006,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6037,6038,0,56,academic/educator,14706,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6038,6039,0,45,other,01060,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
